In [ ]:
import numpy as np
 
class SSQ:
  def __init__(self,choice):   #Initialization
    #self.interarrivals= [0.397937254081555, 0.6279653814829189, 0.4616115729020344, 0.39360057615863536, 0.2755242455477496, 0.5190796474718047, 0.2877595995843199, 1.1117622065163284, 1.657456091026907, 0.24180105045112665, 0.7844480699845842, 0.3763370547682942, 0.41971642336941145, 1.2991270769627203, 0.03684267745642394, 0.04558051455950024, 0.010212793342112475, 0.8937438398074441, 0.7528921159326077, 1.0201571404727754]
    #self.service_times= [4.998788389616904, 2.08681142663519, 0.8046082961968591, 1.9714967699441972, 0.16363673537970563, 1.3278614523096948, 0.20114858754593876, 3.762746584598984, 0.9591754618043763, 0.6962354337909832, 0.3994644590175899, 1.9347312903447555, 0.7918071411940105, 1.0924351189278956, 0.02465914316109906, 1.2497952079533172, 1.2310956811969116, 1.2474123645480066, 3.7412896023633304, 1.4886806462454891]

    
    self.interarrivals= list(np.random.exponential(1.2,70))
    self.service_times= list(np.random.exponential(1.3, 70))
    #print(len(self.interarrivals))
    
    self.clock= 0.0
    
    self.next_arrival=self.interarrivals.pop(0)
    self.next_departure1= float('inf')
    self.next_departure2=float('inf')
    
    self.num_in_queue= 0
    self.times_of_arrivalqueue= []         #store times of arrivals who are waiting in the queue
    self.service_times_in_queue= []   #store service times of waiting customers in the queue
    
    self.total_delay=0.0
    self.num_of_delays= 0.0
    self.area_under_q= 0.0
    self.area_under_bt_s1= 0.0
    self.area_under_bt_s2= 0.0
    
    self.server1_status= 0        #0 for IDLE , 1 for BUSY
    self.server2_status=0
    self.last_event_time=0.0     #we will need to store last event clock time 
 
  def start(self,choice):
    customer_delay = [10, 30, 60]
    for customer in customer_delay:
      print("Initiated for {}th customer delay: ".format(customer))
      print(" ")
      while self.num_of_delays<customer:
        self.timing(choice)
      self.generate_report(customer)

  def timing(self,choice):
    self.clock= min(self.next_arrival,self.next_departure1,self.next_departure2)  #First set clock to minimum time of next event
    self.update_register()
    if self.next_arrival<= self.next_departure1 and self.next_arrival<=self.next_departure2:
      self.arrival()
      print("Arrival at Clock:" +str(self.clock))
 
    elif self.next_departure1<=self.next_arrival and self.next_departure1<=self.next_departure2:
      self.departure1(choice)
      print("Departure from server-1 at: "+str(self.clock))
    
    else:
      self.departure2(choice)
      print("Departure from server-2 at: "+str(self.clock))

    print("Server-1 Status:"+str(self.server1_status))
    print("Server-2 Status:"+str(self.server2_status))
    print("Times of arrivals in Queue: "+ str(self.times_of_arrivalqueue))
    print("Service times in Queue: "+str(self.service_times_in_queue))
    print("Total Delay:" +str(self.total_delay))
    print("Next Arrival Time: "+str(self.next_arrival))
    print("Next Departure Time of Server-1: "+str(self.next_departure1))
    print("Next Departure Time of server-2: "+str(self.next_departure2))
    print("Number of customer delayed: "+str(self.num_of_delays))
    print(" ")
 
  def arrival(self):
    #Schedule next arrival , new_arrival = previous_arrival + inter_arrival time of next customer
    self.next_arrival= self.next_arrival+ self.interarrivals.pop(0)  
    
    if self.server1_status==0:   #server is idle
      self.server1_status= 1    #make server BUSY
      delay=0.0            #so delay is zero
      self.total_delay += delay  
      self.num_of_delays +=1   #increase the number of customers delayed
 
      #schedule next departure, pop the first element of service_times list to get service time of this customer
      self.next_departure1 = self.clock+ self.service_times.pop(0) 
    
    elif self.server1_status==1 and self.server2_status==0:
        self.server2_status= 1    #make server BUSY
        delay=0.0            #so delay is zero
        self.total_delay += delay  
        self.num_of_delays +=1   #increase the number of customers delayed
 
      #schedule next departure, pop the first element of service_times list to get service time of this customer
        self.next_departure2 = self.clock+ self.service_times.pop(0)
    else:     #Server is BUSY
      #increase queue length, this customer will have to wait in the queue
      self.num_in_queue+=1
 
      #store the arrival time and service time of this customer in seperate lists
      self.times_of_arrivalqueue.append(self.clock) 
      self.service_times_in_queue.append(self.service_times.pop(0)) 
 
  def departure1(self,choice): ##depart from server-1
    #check number of customers in the queue
    if self.num_in_queue==0:  #if no customer in the queue
      #make server IDLE 
      self.server1_status= 0
      #schedule next departure= infinity
      self.next_departure1= float('infinity')
    
    else: 
      #if queue not empty, pop one customer, decrease queue length
      self.num_in_queue-=1
      self.num_of_delays+=1
      #AS FIFO, pop first arrival and service time from the queue. IF LIFO we have to pop last arrival and service time
      #For SJF, finf the index of minimum service time from  service_times_in_queue list.
      #Then pop the arrival of that index from times_of_arrivalqueue for delay count and others.  

      if choice==1:
        arrival= self.times_of_arrivalqueue.pop(0)
        self.next_departure1= self.clock+ self.service_times_in_queue.pop(0) 
      if choice==2:
        ind = self.service_times_in_queue.index(min(self.service_times_in_queue))
        arrival= self.times_of_arrivalqueue.pop(ind)
        self.next_departure1= self.clock+ self.service_times_in_queue.pop(ind)
      if choice==3:
        arrival= self.times_of_arrivalqueue.pop(-1)
        self.next_departure1= self.clock+ self.service_times_in_queue.pop(-1)

      delay= self.clock- arrival
      self.total_delay+=delay
 
  def departure2(self,choice): ##depart from server-2
    #check number of customers in the queue
    if self.num_in_queue==0:  #if no customer in the queue
      #make server IDLE 
      self.server2_status= 0
      #schedule next departure= infinity
      self.next_departure2= float('infinity')
    
    else: 
      #if queue not empty, pop one customer, decrease queue length
      self.num_in_queue-=1
      self.num_of_delays+=1
      #AS FIFO, pop first arrival and service time from the queue. IF LIFO we have to pop last arrival and service time
      #For SJF, finf the index of minimum service time from  service_times_in_queue list.
      #Then pop the arrival of that index from times_of_arrivalqueue for delay count and others.  
      if choice==1:
        arrival= self.times_of_arrivalqueue.pop(0)
        self.next_departure2= self.clock+ self.service_times_in_queue.pop(0) 
      if choice==2:
        ind = self.service_times_in_queue.index(min(self.service_times_in_queue))
        arrival= self.times_of_arrivalqueue.pop(ind)
        self.next_departure2= self.clock+ self.service_times_in_queue.pop(ind) 
      if choice==3:
        arrival= self.times_of_arrivalqueue.pop(-1)
        self.next_departure2= self.clock+ self.service_times_in_queue.pop(-1)

      delay= self.clock- arrival
      self.total_delay+=delay
    
 
  def update_register(self):
    time_difference= self.clock - self.last_event_time
    self.area_under_q+= time_difference*self.num_in_queue
    self.area_under_bt_s1+= time_difference*self.server1_status
    self.area_under_bt_s2+= time_difference*self.server2_status    
    self.last_event_time=self.clock;
  
  def generate_report(self,customer):
    print("Customer Delayed: "+str(customer))
    print("Average Delay: "+str(self.total_delay/customer))
    print("Expected number of customers in queue: "+str(self.area_under_q/self.last_event_time))
    print("Expected utilization of server-1: "+str(self.area_under_bt_s1/self.last_event_time))
    print("Expected utilization of server-2: "+str(self.area_under_bt_s2/self.last_event_time))

In [ ]:
print("Enter 1 for FIFO\nEnter 2 for SJF\nEnter 3 for LIFO")
choice = int(input("Enter your choice: "))
print("")
s = SSQ(choice)
s.start(choice)
#self.interarrivals= [0.4,1.2,0.5,1.7,0.2,1.6,0.2,1.4,1.9]  #exponential distribution 
#self.service_times= [2.0,0.7,0.2,1.1,3.7,0.6,0.7,0.9,1.2]  #exponential distribution

Enter 1 for FIFO
Enter 2 for SJF
Enter 3 for LIFO
Enter your choice: 1

Initiated for 10th customer delay: 
 
Arrival at Clock:0.8392349450849862
Server-1 Status:1
Server-2 Status:0
Times of arrivals in Queue: []
Service times in Queue: []
Total Delay:0.0
Next Arrival Time: 0.8586604349811552
Next Departure Time of Server-1: 1.0289061562437025
Next Departure Time of server-2: inf
Number of customer delayed: 1.0
 
Arrival at Clock:0.8586604349811552
Server-1 Status:1
Server-2 Status:1
Times of arrivals in Queue: []
Service times in Queue: []
Total Delay:0.0
Next Arrival Time: 1.181295091762075
Next Departure Time of Server-1: 1.0289061562437025
Next Departure Time of server-2: 4.3733742890901315
Number of customer delayed: 2.0
 
Departure from server-1 at: 1.0289061562437025
Server-1 Status:0
Server-2 Status:1
Times of arrivals in Queue: []
Service times in Queue: []
Total Delay:0.0
Next Arrival Time: 1.181295091762075
Next Departure Time of Server-1: inf
Next Departure Time of server-2

0
